## SQLite in Python - A Comprehensive Guide

**Introduction:**

SQLite is a self-contained, serverless, zero-configuration, transactional SQL database engine. It's a popular choice for embedded systems, mobile applications, and desktop software due to its simplicity and portability.  Python's built-in `sqlite3` module provides a straightforward interface for interacting with SQLite databases.

**Course Outline:**

1.  **Setting Up and Connecting:**
    *   Introduction to SQLite
    *   The `sqlite3` Module
    *   Creating and Connecting to Databases (In-Memory and File-Based)
    *   Cursor Objects

2.  **Creating Tables and Basic CRUD Operations:**
    *   Data Types in SQLite
    *   `CREATE TABLE` Syntax
    *   `INSERT` Statements
    *   `SELECT` Statements (Basic Queries)
    *   `WHERE` Clause (Filtering)
    *   `UPDATE` Statements
    *   `DELETE` Statements
    *   Error Handling (Basic `try-except` Blocks)

3.  **Advanced Querying and Data Manipulation:**
    *   `ORDER BY` (Sorting)
    *   `LIMIT` (Restricting Results)
    *   `GROUP BY` and Aggregate Functions (`COUNT`, `SUM`, `AVG`, `MIN`, `MAX`)
    *   `DISTINCT` (Unique Values)
    *   `LIKE` and Wildcards (Pattern Matching)
    *   `BETWEEN` (Range Queries)
    *   `IN` and `NOT IN` (Set Membership)
    *   `NULL` Values and `IS NULL` / `IS NOT NULL`
    *   Date and Time Handling

4.  **Working with Multiple Tables (Joins):**
    *   Understanding Relationships (One-to-One, One-to-Many, Many-to-Many)
    *   Foreign Keys
    *   `INNER JOIN`
    *   `LEFT JOIN`
    *   `RIGHT JOIN` (Not directly supported in SQLite, but emulated)
    *   `FULL OUTER JOIN` (Not directly supported in SQLite, but emulated)

5.  **Transactions and Data Integrity:**
    *   What are Transactions?
    *   `BEGIN TRANSACTION`, `COMMIT`, and `ROLLBACK`
    *   Ensuring Atomicity
    *   Using `with` Statements for Automatic Transaction Management

6.  **Parameterized Queries and Security (Preventing SQL Injection):**
    *   The Danger of String Concatenation
    *   Using Placeholders (`?` and Named Placeholders)
    *   `execute()` with Parameter Tuples/Dictionaries

7.  **Working with BLOBs (Binary Data):**
    *   Storing Images, Files, etc.
    *   Reading and Writing BLOB Data

8.  **User-Defined Functions (UDFs) and Aggregates:**
    *   Creating Custom Functions in Python
    *   Registering UDFs with SQLite
    *   Creating Custom Aggregate Functions

9. **Advanced SQLite Features:**
   *   Row Factory
   *   Attach Database

**Detailed Modules with Code Examples:**

**(1) Setting Up and Connecting:**

```python
import sqlite3

# Connect to a database file (creates it if it doesn't exist)
conn = sqlite3.connect('mydatabase.db')
print("Opened database successfully (file-based)")

# Connect to an in-memory database (temporary, data lost when connection closes)
mem_conn = sqlite3.connect(':memory:')
print("Opened database successfully (in-memory)")

# Create a cursor object (used to execute SQL commands)
cursor = conn.cursor()
mem_cursor = mem_conn.cursor()

#Close connections (Important to free resources)
conn.close()
mem_conn.close()

```

**(2) Creating Tables and Basic CRUD Operations:**

```python
import sqlite3

conn = sqlite3.connect('library.db')
cursor = conn.cursor()

# Create a 'books' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS books (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT NOT NULL,
        author TEXT,
        publication_year INTEGER,
        genre TEXT,
        price REAL
    )
''')

# Insert data
cursor.execute("INSERT INTO books (title, author, publication_year, genre, price) VALUES (?, ?, ?, ?, ?)",
               ("The Hitchhiker's Guide to the Galaxy", "Douglas Adams", 1979, "Science Fiction", 12.99))
cursor.execute("INSERT INTO books (title, author, publication_year, genre, price) VALUES (?, ?, ?, ?, ?)",
               ("Pride and Prejudice", "Jane Austen", 1813, "Romance", 9.99))

conn.commit() # Commit the changes

# Select all books
cursor.execute("SELECT * FROM books")
all_books = cursor.fetchall()
print("All Books:", all_books)

# Select books published after 1900
cursor.execute("SELECT * FROM books WHERE publication_year > 1900")
modern_books = cursor.fetchall()
print("Modern Books:", modern_books)

# Update the price of a book
cursor.execute("UPDATE books SET price = ? WHERE title = ?", (14.99, "The Hitchhiker's Guide to the Galaxy"))
conn.commit()

# Delete a book
cursor.execute("DELETE FROM books WHERE title = ?", ("Pride and Prejudice",)) #Tuple even with 1 element
conn.commit()

# Basic Error Handling
try:
    cursor.execute("SELECT * FROM non_existent_table") # This will cause an error
except sqlite3.OperationalError as e:
    print(f"Database error: {e}")

conn.close()
```

**(3) Advanced Querying and Data Manipulation:**

```python
import sqlite3

conn = sqlite3.connect('library.db')
cursor = conn.cursor()

# --- Insert more data for demonstration (assuming previous table creation) ---
cursor.execute("INSERT INTO books (title, author, publication_year, genre, price) VALUES (?, ?, ?, ?, ?)",
               ("1984", "George Orwell", 1949, "Dystopian", 10.50))
cursor.execute("INSERT INTO books (title, author, publication_year, genre, price) VALUES (?, ?, ?, ?, ?)",
               ("To Kill a Mockingbird", "Harper Lee", 1960, "Fiction", 11.00))
cursor.execute("INSERT INTO books (title, author, publication_year, genre, price) VALUES (?, ?, ?, ?, ?)",
               ("The Lord of the Rings", "J.R.R. Tolkien", 1954, "Fantasy", 25.00))
cursor.execute("INSERT INTO books (title, author, publication_year, genre, price) VALUES (?, ?, ?, ?, ?)",
               ("Brave New World", "Aldous Huxley", 1932, "Dystopian", 9.75))
cursor.execute("INSERT INTO books (title, author, publication_year, genre, price) VALUES (?, ?, ?, ?, ?)",
              ("The Great Gatsby", "F. Scott Fitzgerald", 1925, "Fiction", 10.00))
cursor.execute("INSERT INTO books (title, author, publication_year, genre, price) VALUES (?, ?, ?, ?, ?)",
               ("One Hundred Years of Solitude", "Gabriel Garcia Marquez", 1967, "Magical Realism", 14.00))
conn.commit()

# ORDER BY (Sort by publication year, then by title)
cursor.execute("SELECT * FROM books ORDER BY publication_year DESC, title ASC")
print("\nSorted Books:", cursor.fetchall())

# LIMIT
cursor.execute("SELECT * FROM books LIMIT 3")
print("\nFirst 3 Books:", cursor.fetchall())

# GROUP BY and Aggregate Functions
cursor.execute("SELECT genre, COUNT(*) AS num_books, AVG(price) AS avg_price FROM books GROUP BY genre")
print("\nGenre Statistics:", cursor.fetchall())

# DISTINCT
cursor.execute("SELECT DISTINCT author FROM books")
print("\nUnique Authors:", cursor.fetchall())

# LIKE and Wildcards
cursor.execute("SELECT * FROM books WHERE title LIKE 'The%'")  # Starts with "The"
print("\nBooks Starting with 'The':", cursor.fetchall())

cursor.execute("SELECT * FROM books WHERE author LIKE '%Lee%'") # Contains "Lee"
print("Books by Authors with 'Lee':", cursor.fetchall())

# BETWEEN
cursor.execute("SELECT * FROM books WHERE publication_year BETWEEN 1950 AND 1970")
print("\nBooks Published Between 1950 and 1970:", cursor.fetchall())

# IN and NOT IN
cursor.execute("SELECT * FROM books WHERE genre IN ('Dystopian', 'Fantasy')")
print("\nDystopian and Fantasy Books:", cursor.fetchall())

# NULL Values (Let's add a book with a missing price)
cursor.execute("INSERT INTO books (title, author, publication_year, genre) VALUES (?, ?, ?, ?)",
               ("The Catcher in the Rye", "J.D. Salinger", 1951, "Fiction"))  # No price
conn.commit()

cursor.execute("SELECT * FROM books WHERE price IS NULL")
print("\nBooks with Missing Price:", cursor.fetchall())
cursor.execute("SELECT * FROM books WHERE price IS NOT NULL")
print("Books with Price:", cursor.fetchall())

# Date and Time (using strings for simplicity, but you can use INTEGER as Unix timestamp)
cursor.execute("ALTER TABLE books ADD COLUMN publication_date TEXT") #Add the new column

#Insert with dates
cursor.execute("UPDATE books SET publication_date = '1949-06-08' WHERE title = '1984'")
cursor.execute("UPDATE books SET publication_date = '1960-07-11' WHERE title = 'To Kill a Mockingbird'")
cursor.execute("UPDATE books SET publication_date = '1954-07-29' WHERE title = 'The Lord of the Rings'")
conn.commit()

cursor.execute("SELECT * FROM books WHERE publication_date < '1960-01-01'")
print("\nBooks published before 1960:", cursor.fetchall())

conn.close()
```

**(4) Working with Multiple Tables (Joins):**

```python
import sqlite3

conn = sqlite3.connect('library.db')
cursor = conn.cursor()

# Create a 'customers' table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS customers (
        customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        email TEXT
    )
''')

# Create a 'borrowings' table (linking customers and books)
cursor.execute('''
    CREATE TABLE IF NOT EXISTS borrowings (
        borrowing_id INTEGER PRIMARY KEY AUTOINCREMENT,
        customer_id INTEGER,
        book_id INTEGER,
        borrow_date TEXT,
        return_date TEXT,
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
        FOREIGN KEY (book_id) REFERENCES books(id)
    )
''')
conn.commit()

# Insert some customers
cursor.execute("INSERT INTO customers (name, email) VALUES (?, ?)", ("Alice Smith", "alice@example.com"))
cursor.execute("INSERT INTO customers (name, email) VALUES (?, ?)", ("Bob Johnson", "bob@example.com"))
conn.commit()

# Insert some borrowings
cursor.execute("INSERT INTO borrowings (customer_id, book_id, borrow_date) VALUES (?, ?, ?)", (1, 1, "2023-10-26")) # Alice borrows "The Hitchhiker's Guide"
cursor.execute("INSERT INTO borrowings (customer_id, book_id, borrow_date) VALUES (?, ?, ?)", (2, 3, "2023-10-27"))  # Bob borrows "The Lord of the Rings"
cursor.execute("INSERT INTO borrowings (customer_id, book_id, borrow_date, return_date) VALUES (?, ?, ?,?)", (1, 4, "2023-10-27", "2023-11-15"))  # Bob borrows "Brave new world"
conn.commit()

# INNER JOIN (Customers who have borrowed books, and the book details)
cursor.execute('''
    SELECT customers.name, customers.email, books.title, books.author, borrowings.borrow_date
    FROM borrowings
    INNER JOIN customers ON borrowings.customer_id = customers.customer_id
    INNER JOIN books ON borrowings.book_id = books.id
''')
print("\nInner Join (Borrowed Books):", cursor.fetchall())

# LEFT JOIN (All customers, and any books they have borrowed)
cursor.execute('''
    SELECT customers.name, customers.email, books.title
    FROM customers
    LEFT JOIN borrowings ON customers.customer_id = borrowings.customer_id
    LEFT JOIN books ON borrowings.book_id = books.id
''')
print("\nLeft Join (All Customers and Borrowed Books):", cursor.fetchall())

#RIGHT JOIN and FULL OUTER JOIN simulation
cursor.execute('''
    SELECT customers.name, customers.email, books.title
    FROM borrowings
    LEFT JOIN customers ON borrowings.customer_id = customers.customer_id
    LEFT JOIN books ON borrowings.book_id = books.id
    
    UNION ALL
    
    SELECT customers.name, customers.email, books.title
    FROM customers
    LEFT JOIN borrowings ON customers.customer_id = borrowings.customer_id
    LEFT JOIN books ON borrowings.book_id = books.id
    WHERE borrowings.borrowing_id IS NULL
''')
print("\nFull outer Join simulation (All Customers and Borrowed Books):", cursor.fetchall())

conn.close()
```

**(5) Transactions and Data Integrity:**

```python
import sqlite3

conn = sqlite3.connect('bank.db')
cursor = conn.cursor()

# Create accounts table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS accounts (
        account_id INTEGER PRIMARY KEY,
        name TEXT,
        balance REAL
    )
''')
conn.commit()

# Insert initial accounts
cursor.execute("INSERT INTO accounts (name, balance) VALUES (?, ?)", ("Alice", 1000.0))
cursor.execute("INSERT INTO accounts (name, balance) VALUES (?, ?)", ("Bob", 500.0))
conn.commit()

def transfer_funds(from_account, to_account, amount):
    try:
        # Start a transaction
        conn.execute("BEGIN TRANSACTION") #Not needed when using "with"

        # Withdraw from the source account
        cursor.execute("UPDATE accounts SET balance = balance - ? WHERE name = ?", (amount, from_account))

        # Check if sufficient balance
        cursor.execute("SELECT balance FROM accounts WHERE name = ?", (from_account,))
        from_balance = cursor.fetchone()[0]
        if from_balance < 0:
            raise ValueError("Insufficient funds")


        # Deposit to the destination account
        cursor.execute("UPDATE accounts SET balance = balance + ? WHERE name = ?", (amount, to_account))

        # Commit the transaction
        conn.commit() #Not needed when using "with"
        print(f"Transferred ${amount} from {from_account} to {to_account}.")

    except Exception as e:
        # Rollback the transaction if any error occurs
        conn.rollback()
        print(f"Transaction failed: {e}")

# --- Using 'with' statement (automatic transaction management) ---
def transfer_funds_with(from_account, to_account, amount):
    with conn: # Automatically handles BEGIN, COMMIT, and ROLLBACK
        cursor.execute("UPDATE accounts SET balance = balance - ? WHERE name = ?", (amount, from_account))

        # Check if sufficient balance
        cursor.execute("SELECT balance FROM accounts WHERE name = ?", (from_account,))
        from_balance = cursor.fetchone()[0]

        if from_balance < 0:
           raise ValueError("Insufficient funds")

        cursor.execute("UPDATE accounts SET balance = balance + ? WHERE name = ?", (amount, to_account))
    print(f"Transferred ${amount} from {from_account} to {to_account}.")


#Successful transaction
transfer_funds("Alice", "Bob", 200.0)

#This transaction will fail
transfer_funds("Alice", "Bob", 2000.0)

#Successful transaction using with
transfer_funds_with("Alice", "Bob", 100.0)

#Check the final amounts
cursor.execute("SELECT * from accounts")
print("\nFinal balances", cursor.fetchall())
conn.close()

```

**(6) Parameterized Queries and Security (Preventing SQL Injection):**

```python
import sqlite3

conn = sqlite3.connect('library.db')
cursor = conn.cursor()

# --- BAD PRACTICE (Vulnerable to SQL Injection) ---
# user_input = "'; DROP TABLE books; --"  # Malicious input
# cursor.execute(f"SELECT * FROM books WHERE author = '{user_input}'") # NEVER DO THIS!

# --- GOOD PRACTICE (Parameterized Query) ---
user_input = "Douglas Adams"
cursor.execute("SELECT * FROM books WHERE author = ?", (user_input,))  # Safe
print("\nBooks by Douglas Adams (Parameterized):", cursor.fetchall())

# Named Placeholders
user_input = "Dystopian"
cursor.execute("SELECT * FROM books WHERE genre = :genre", {"genre": user_input}) #Safe
print("\nDystopian books (Named Placeholders):", cursor.fetchall())
conn.close()
```

**(7) Working with BLOBs (Binary Data):**

```python
import sqlite3

conn = sqlite3.connect('images.db')
cursor = conn.cursor()

# Create a table to store images
cursor.execute('''
    CREATE TABLE IF NOT EXISTS images (
        id INTEGER PRIMARY KEY,
        name TEXT,
        data BLOB
    )
''')
conn.commit()

# Function to insert an image
def insert_image(image_path, image_name):
    try:
        with open(image_path, 'rb') as file:  # Open the image file in binary read mode
            image_data = file.read()
            cursor.execute("INSERT INTO images (name, data) VALUES (?, ?)", (image_name, image_data))
            conn.commit()
            print(f"Image '{image_name}' inserted successfully.")
    except FileNotFoundError:
        print(f"Error: File not found at {image_path}")
    except Exception as e:
        print(f"Error inserting image: {e}")


# Function to retrieve and save an image
def retrieve_image(image_id, output_path):
    cursor.execute("SELECT data FROM images WHERE id = ?", (image_id,))
    image_data = cursor.fetchone()

    if image_data:
        with open(output_path, 'wb') as file:  # Open the output file in binary write mode
            file.write(image_data[0])
        print(f"Image retrieved and saved to {output_path}")
    else:
        print(f"Image with ID {image_id} not found.")

# --- Example Usage ---
# (Assume you have an image file named 'example.png' in the same directory)

insert_image('example.png', 'Example Image')
retrieve_image(1, 'retrieved_image.png')

conn.close()
```

**(8) User-Defined Functions (UDFs) and Aggregates:**

```python
import sqlite3
import math

conn = sqlite3.connect('udfs.db')
cursor = conn.cursor()

# Create a sample table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS numbers (
        id INTEGER PRIMARY KEY,
        value REAL
    )
''')

cursor.execute("INSERT INTO numbers (value) VALUES (25)")
cursor.execute("INSERT INTO numbers (value) VALUES (16)")
cursor.execute("INSERT INTO numbers (value) VALUES (9)")
cursor.execute("INSERT INTO numbers (value) VALUES (4)")
cursor.execute("INSERT INTO numbers (value) VALUES (NULL)")
conn.commit()

# --- User-Defined Function (UDF) ---
def square_root(value):
    if value is None:
        return None
    return math.sqrt(value)

# Register the UDF with SQLite
conn.create_function("sqrt", 1, square_root)  # Name, number of arguments, function

cursor.execute("SELECT value, sqrt(value) FROM numbers")
print("\nSquare Roots:", cursor.fetchall())

# --- User-Defined Aggregate ---

class MyAverage:
    def __init__(self):
        self.total = 0
        self.count = 0

    def step(self, value):
        if value is not None:
            self.total += value
            self.count += 1

    def finalize(self):
        if self.count == 0:
            return None
        return self.total / self.count

#Register the Aggregate with SQLite
conn.create_aggregate("myavg", 1, MyAverage)

cursor.execute("SELECT myavg(value) FROM numbers")
print("\nMy Custom Average:", cursor.fetchone()[0])

conn.close()
```

**(9) Advanced SQLite Features**
```python
import sqlite3

conn = sqlite3.connect('mydatabase.db')
cursor = conn.cursor()

#Row Factory: Access columns by name
conn.row_factory = sqlite3.Row
cursor = conn.cursor() #Need to recreate cursor after changing row factory
cursor.execute("SELECT * FROM books")
for row in cursor.fetchall():
    print(f"Title {row['title']}, Author: {row['author']}")

#Attach database: Attach another database
conn.execute("ATTACH DATABASE 'library.db' AS library")

#Now you can query library.db using "library" as prefix
cursor.execute("SELECT * FROM library.books")
print(cursor.fetchall())
#Detach database
conn.execute("DETACH DATABASE library")

conn.close()

```
This detailed course provides a comprehensive guide to using SQLite with Python. By working through the modules and code examples, you'll gain the skills needed to effectively manage data in your Python applications using SQLite.  Remember to experiment, adapt the code to your own projects, and consult the official SQLite and `sqlite3` documentation for further details.
